# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [79]:
!pip install chromadb langchain langchain_community langchain_openai

In [80]:
from google.colab import userdata
import os

# Get key from Colab secrets
key = userdata.get("OPENAI_API_KEY")
print("Secret from Colab starts with:", repr(key)[:10])  # quick sanity check

# Put it into the env var that langchain_openai expects
os.environ["OPENAI_API_KEY"] = key

print("Env var OPENAI_API_KEY now:", os.environ["OPENAI_API_KEY"][:5])  # should show 'sk-' or 'op-'


Secret from Colab starts with: 'sk-proj-a
Env var OPENAI_API_KEY now: sk-pr


In [81]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [82]:
llm = OpenAI(temperature=0)

In [83]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = "/content/sonnets.txt"

In [85]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [86]:
sonnets = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [87]:
from langchain_community.document_loaders import WebBaseLoader

In [88]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [89]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [90]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [95]:
tools = [
    Tool(
        name="shakespeare-sonnets",
        func=sonnets.run,  # your RetrievalQA or retriever chain
        description=(
            "Useful for answering questions about Shakespeare's sonnets: "
            "their exact wording, which sonnet says what, themes, imagery, "
            "and quotes. Input should be a fully formed question."
        ),
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="Useful for when you need to answer questions about ruff (a Python linter). Input should be a fully formed question.",
    ),
]


In [96]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [97]:
agent.invoke("Which sonnet starts with 'Shall I compare thee to a summer's day'?")



> Entering new AgentExecutor chain...
 I should use the shakespeare-sonnets tool to find the sonnet.
Action: shakespeare-sonnets
Action Input: 'Shall I compare thee to a summer's day'
Observation:  This line is the first line of Shakespeare's Sonnet 18, in which the speaker is contemplating whether or not to compare his beloved to a summer's day.
Thought: I should use the shakespeare-sonnets tool to find the sonnet number.
Action: shakespeare-sonnets
Action Input: Sonnet 18
Observation:  Sonnet 18 is a poem written by William Shakespeare in which the speaker compares the beauty of a person to a summer's day, and declares that their beauty will never fade because it will be immortalized in the poem.
Thought: I now know the final answer.
Final Answer: Sonnet 18.

> Finished chain.


{'input': "Which sonnet starts with 'Shall I compare thee to a summer's day'?",
 'output': 'Sonnet 18.'}

In [98]:
agent.invoke(
    "Where in the sonnets does Shakespeare talk about love surviving beyond death? "
    "Quote the most relevant lines and explain them."
)



> Entering new AgentExecutor chain...
 I should use the shakespeare-sonnets tool to find the sonnet that talks about love surviving beyond death.
Action: shakespeare-sonnets
Action Input: "love surviving beyond death"
Observation:  Yes, the speaker believes that their love will continue even after death.
Thought: Now I need to find the most relevant lines and explain them.
Action: shakespeare-sonnets
Action Input: "love surviving beyond death"
Observation:  Yes, the speaker believes that their love will continue even after death.
Thought: The most relevant lines are likely to be towards the end of the sonnet.
Action: shakespeare-sonnets
Action Input: "love surviving beyond death"
Observation:  Yes, the speaker believes that their love will continue even after death.
Thought: The most relevant lines are likely to be towards the end of the sonnet.
Action: shakespeare-sonnets
Action Input: "love surviving beyond death"
Observation:  Yes, the speaker believes that their love will continu

{'input': 'Where in the sonnets does Shakespeare talk about love surviving beyond death? Quote the most relevant lines and explain them.',
 'output': 'The final answer is that Shakespeare talks about love surviving beyond death in Sonnet 116, with the most relevant lines being "Love\'s not Time\'s fool, though rosy lips and cheeks / Within his bending sickle\'s compass come" and "If this be error and upon me proved, / I never writ, nor no man ever loved." These lines suggest that love is not subject to the passing of time and will continue even after death, and the speaker is willing to stake their reputation on this belief.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [99]:
tools = [
    Tool(
        name="Shakespeare Sonnets QA System",
        func=sonnets.run,  # your RetrievalQA chain for the sonnets
        description=(
            "Useful for answering questions about Shakespeare's sonnets. "
            "Use this for questions about quotes, which sonnet says what, themes, "
            "interpretation of passages, locating lines, or anything directly about "
            "the content of the sonnets. Input should be a fully formed question."
        ),
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,  # your Ruff documentation retriever
        description=(
            "Useful for answering questions about Ruff, the Python linter. "
            "Use this for questions about Ruff rules, lint errors, configuration, "
            "CLI usage, settings, pyproject.toml, or how to fix specific lint issues. "
            "Input should be a fully formed question."
        ),
        return_direct=True,
    ),
]


In [100]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [101]:
agent.invoke(
    "Find two sonnets where Shakespeare is pessimistic about love and two where he is hopeful, and summarize each."
)



> Entering new AgentExecutor chain...
 I should use the Shakespeare Sonnets QA System to locate the sonnets and then read them to determine Shakespeare's attitude towards love.
Action: Shakespeare Sonnets QA System
Action Input: "What are two sonnets where Shakespeare is pessimistic about love?"
Observation:  Sonnets XLVI and LXIV


> Finished chain.


{'input': 'Find two sonnets where Shakespeare is pessimistic about love and two where he is hopeful, and summarize each.',
 'output': ' Sonnets XLVI and LXIV'}

In [102]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff is a newer linter and may have more advanced features compared to flake8.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff offers a larger rule set and automatic fixing of lint violations, while also being able to replace other tools such as Black, isort, yesqa, eradicate, and most of the rules implemented in pyupgrade. It also does not require the installation of Rust.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': ' Ruff offers a larger rule set and automatic fixing of lint violations, while also being able to replace other tools such as Black, isort, yesqa, eradicate, and most of the rules implemented in pyupgrade. It also does not require the installation of Rust.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [110]:

tools = [
    Tool(
        name="Shakespeare Sonnets QA System",
        func=sonnets.run,  # your RetrievalQA chain over the sonnets vector store
        description=(
            "Useful for answering questions about Shakespeare's sonnets. "
            "Use this for questions about specific lines, which sonnet says what, "
            "themes, metaphors, and interpretations. "
            "Input should be a fully formed question, not using pronouns like "
            "'he', 'it', or 'they' that depend on earlier context."
        ),
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,  # your RetrievalQA chain over Ruff docs
        description=(
            "Useful for answering questions about Ruff, the Python linter. "
            "Use this for questions about its rules, configuration, CLI usage, "
            "pyproject.toml settings, and how to fix lint errors. "
        ),
    )
  ]


In [111]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [112]:
agent.invoke(
    "What metaphors does Shakespeare use for love in the sonnets?"
)



> Entering new AgentExecutor chain...
 We should use the Shakespeare Sonnets QA System to answer this question.
Action: Shakespeare Sonnets QA System
Action Input: "What metaphors does Shakespeare use for love in the sonnets?"
Observation:  Shakespeare uses several metaphors for love in the sonnets, including love as a force that can be strengthened or weakened, love as a commodity that can be bought and sold, love as a season that changes over time, and love as a bond between two people.
Thought: This is helpful information, but I want to know more about specific examples of these metaphors.
Action: Shakespeare Sonnets QA System
Action Input: "Can you give me an example of Shakespeare using love as a force in the sonnets?"
Observation:  In Sonnet LVII, Shakespeare compares love to a force that can be renewed and strengthened over time, despite any initial weaknesses or changes. He also uses the metaphor of love as a growing plant, showing how it can continue to flourish and evolve.


{'input': 'What metaphors does Shakespeare use for love in the sonnets?',
 'output': 'Shakespeare uses several metaphors for love in the sonnets, including love as a force that can be strengthened or weakened, love as a commodity that can be bought and sold, love as a season that changes over time, and love as a bond between two people.'}